In [59]:
# the purpose of this notebook is to analyze the defi llama protocols and find what percentage of overall TVL is in forks
# 1. load the json file into a pandas dataframe
# 2. filter the dataframe to only include protocols with TVL > 0
# 3. filter the dataframe to only include protocols with a forkedFrom value
# 4. calculate the total TVL of all protocols


# Future analysis
# cut by number of users / txs instead of TVL as skewed by whales
# look at growth
# look at what chain they're on


In [60]:
# CREATE OUR DATASET FROM DEFI LLAMA SOURCE

import pandas as pd
import json
import time
import os

# load json
df = pd.read_json('defillamaprotocols.json')

print (str(len(df)) + " protocols to start with")

# filter df to only include protocols with TVL > 0
df = df[df['tvl'] > 0]

print (str(len(df)) + " protocols with TVL > 0")

results = pd.DataFrame(columns=['category', 'name', 'forkedFrom', 'tvl', 'chain'])

for index, row in df.iterrows():
  new_row = pd.DataFrame({'category': [row['category']], 'name': [row['name']], 'forkedFrom': [row['forkedFrom']], 'tvl': [row['tvl']], 'chain': [row['chain']]})
  results = pd.concat([results, new_row], ignore_index=True)


# tidy this up as it's inconsistent otherwise
results['forkedFrom'] = results['forkedFrom'].apply(lambda x: x if x != [] else None)

# save results as csv
results.to_csv('protocols_all_small.csv', index=False)


2875 protocols to start with
2401 protocols with TVL > 0


In [58]:
# GET SUMMARY STATS ON FORKS PER CATEGORY

df = pd.read_csv('protocols_all_small.csv')

# for each category, report:
# 1. total TVL
# 2. total TVL in forks
# 3. percentage of TVL in forks
# 4. number of protocols in category
# 5. number of protocols in category that are forks
# 6. percentage of protocols in category that are forks
# 7. average TVL of protocols in category
# 8. average TVL of protocols in category that are forks

forks_summary = pd.DataFrame(columns=['category', 'total_tvl', 'total_tvl_forks', 'percent_forks', 'num_protocols', 'num_protocols_forks', 'percent_protocols_forks', 'avg_tvl', 'avg_tvl_forks'])

# loop through each category and find the total TVL of all protocols in that category
for category in df['category'].unique():
    # create a new DataFrame with the row data
    new_row = pd.DataFrame({
        'category': [category], 
        'total_tvl': [df[df['category'] == category]['tvl'].sum()], 
        'total_tvl_forks': [df[(df['category'] == category) & (df['forkedFrom'].notna())]['tvl'].sum()], 
        'percent_forks': [df[(df['category'] == category) & (df['forkedFrom'].notna())]['tvl'].sum() / df[df['category'] == category]['tvl'].sum()], 
        'num_protocols': [len(df[df['category'] == category])], 
        'num_protocols_forks': [len(df[(df['category'] == category) & (df['forkedFrom'].notna())])], 
        'percent_protocols_forks': [len(df[(df['category'] == category) & (df['forkedFrom'].notna())]) / len(df[df['category'] == category])], 
        'avg_tvl': [df[df['category'] == category]['tvl'].mean()], 
        'avg_tvl_forks': [df[(df['category'] == category) & (df['forkedFrom'].notna())]['tvl'].mean()]
    })

    # concatenate the new row with the existing results DataFrame
    forks_summary = pd.concat([results, new_row], ignore_index=True)

# save results as csv
forks_summary.to_csv('forks_summary.csv', index=False)


In [8]:
# Pretty print summary stats on forks per category

df = pd.read_csv('forks_summary.csv')

df['percent_forks'] = df['percent_forks'].apply(lambda x: str(round(x * 100, 2)) + "%")
df['percent_protocols_forks'] = df['percent_protocols_forks'].apply(lambda x: str(round(x * 100, 2)) + "%")

# format total_tvl, total_tvl_forks, avg_tvl and avg_tvl_forks as millions of $ with 2 decimal places
df['total_tvl'] = df['total_tvl'].apply(lambda x: "$" + str(round(x / 1000000, 2)) + "M")
df['total_tvl_forks'] = df['total_tvl_forks'].apply(lambda x: "$" + str(round(x / 1000000, 2)) + "M")
df['avg_tvl'] = df['avg_tvl'].apply(lambda x: "$" + str(round(x / 1000000, 2)) + "M")
df['avg_tvl_forks'] = df['avg_tvl_forks'].apply(lambda x: "$" + str(round(x / 1000000, 2)) + "M")

,category,total_tvl,total_tvl_forks,percent_forks,num_protocols,num_protocols_forks,percent_protocols_forks,avg_tvl,avg_tvl_forks
0,CEX,$98808.44M,$0.0M,0.0%,26,0,0.0%,$3800.32M,$nanM
1,Liquid Staking,$19559.06M,$109.39M,0.56%,86,4,4.65%,$227.43M,$27.35M
2,CDP,$9311.14M,$143.9M,1.55%,89,26,29.21%,$104.62M,$5.53M
3,Chain,$6488.77M,$0.0M,0.0%,7,1,14.29%,$926.97M,$0.0M
4,Bridge,$10431.25M,$190.93M,1.83%,41,2,4.88%,$254.42M,$95.47M
5,Dexes,$16444.37M,$4120.06M,25.05%,818,501,61.25%,$20.1M,$8.22M
6,Lending,$13951.75M,$1768.94M,12.68%,237,106,44.73%,$58.87M,$16.69M
7,Yield,$5044.37M,$38.53M,0.76%,351,28,7.98%,$14.37M,$1.38M
8,Services,$2348.45M,$0.0M,0.0%,48,1,2.08%,$48.93M,$0.0M
9,Derivatives,$1348.02M,$41.08M,3.05%,108,32,29.63%,$12.48M,$1.28M


In [61]:
# Find what percentage of tvl of each category is covered by the largest n protocols by tvl and their respective forks

df = pd.read_csv('protocols_all_small.csv')

n = 5

results = pd.DataFrame(columns=['category', 'largest_protocols', 'percent_tvl_largest_protocols', 'percent_tvl_largest_protocols_and_forks', 'total_tvl_largest_protocols', 'total_tvl_largest_protocols_and_forks'])

for category in df['category'].unique():

    # find the n largest protocols by TVL
    largest_protocols = df[df['category'] == category].nlargest(n, 'tvl')
    # find the total TVL of the n largest protocols
    total_tvl_largest_protocols = largest_protocols['tvl'].sum()

    # get protocols in this category where forkedFrom is not empty 
    mask = df[(df['category'] == category) & (df['forkedFrom'].notna())]
    # get protocols in this category where forkedFrom is not empty and contains any of the n largest protocols
    mask = mask[mask['forkedFrom'].apply(lambda x: any(item for item in largest_protocols['name'].tolist() if item in x))]
    fork_protocols = mask

    # if fork_protocols is not empty, find the sum of tvl of all protocols in fork_protocols
    if len(fork_protocols) > 0:
        total_tvl_fork_protocols = fork_protocols['tvl'].sum()
        fork_protocol_names = fork_protocols['name'].tolist()
    else:
        total_tvl_fork_protocols = 0
        fork_protocol_names = []

    # calculate the percentage of TVL in this category that is in the n largest protocols
    percent_tvl_largest_protocols = total_tvl_largest_protocols / df[df['category'] == category]['tvl'].sum()
    # calculate the percentage of TVL in this category that is in the n largest protocols and their forks
    percent_tvl_largest_protocols_and_forks = (total_tvl_largest_protocols + total_tvl_fork_protocols) / df[df['category'] == category]['tvl'].sum()

    # calculate the total_tvl_largest_protocols_and_forks
    total_tvl_largest_protocols_and_forks = total_tvl_largest_protocols + total_tvl_fork_protocols

    # fill results
    new_row = pd.DataFrame({
        'category': category,
        'largest_protocols': [largest_protocols['name'].tolist()],
        'percent_tvl_largest_protocols': [percent_tvl_largest_protocols],
        'percent_tvl_largest_protocols_and_forks': [percent_tvl_largest_protocols_and_forks],
        'total_tvl_largest_protocols': [total_tvl_largest_protocols],
        'total_tvl_largest_protocols_and_forks': [total_tvl_largest_protocols_and_forks],
        'fork_protocols': [fork_protocol_names],
        'total_tvl_fork_protocols': [total_tvl_fork_protocols]
    })
    results = pd.concat([results, new_row], ignore_index=True)


    # SANITY CHECKS
    print (category)
    print (largest_protocols['name'].tolist())

    for index, row in fork_protocols.iterrows():
        
        # print out the labelled values in fork_protocols
        print (row['category'] + ": " + row['name'] + ": " + row['forkedFrom'] + ": " + str(row['tvl']))
    
    print ('\n\n')


# save results as csv
results.to_csv('top5.csv', index=False)

# pretty print results, all tvl values in millions of $
df = pd.read_csv('top5.csv')

df['percent_tvl_largest_protocols'] = df['percent_tvl_largest_protocols'].apply(lambda x: str(round(x * 100, 2)) + "%")
df['percent_tvl_largest_protocols_and_forks'] = df['percent_tvl_largest_protocols_and_forks'].apply(lambda x: str(round(x * 100, 2)) + "%")
df['total_tvl_largest_protocols'] = df['total_tvl_largest_protocols'].apply(lambda x: "$" + str(round(x / 1000000, 2)) + "M")
df['total_tvl_largest_protocols_and_forks'] = df['total_tvl_largest_protocols_and_forks'].apply(lambda x: "$" + str(round(x / 1000000, 2)) + "M")
df['total_tvl_fork_protocols'] = df['total_tvl_fork_protocols'].apply(lambda x: "$" + str(round(x / 1000000, 2)) + "M")


df



CEX
['Binance CEX', 'OKX', 'Bitfinex', 'Crypto-com', 'Bybit']



Liquid Staking
['Lido', 'Coinbase Wrapped Staked ETH', 'Rocket Pool', 'Frax Ether', 'StakeWise']
Liquid Staking: OKC Liquid Staking: ['Lido']: 95496145.96332309



CDP
['MakerDAO', 'JustStables', 'Liquity', 'Abracadabra', 'Lybra Finance']
CDP: Helio Protocol: ['MakerDAO']: 48073492.45624114
CDP: Gravita Protocol: ['Liquity']: 20233047.829031557
CDP: DeFi Franc: ['Liquity']: 17916915.097387068
CDP: Vesta Finance: ['Liquity']: 15825425.39945212
CDP: Yeti Finance: ['Liquity']: 12774001.652068302
CDP: Ethos Reserve: ['Liquity']: 5966495.617225461
CDP: FLRLoans: ['Liquity']: 1314058.5736362722
CDP: Davos Protocol: ['MakerDAO']: 508930.1733956234
CDP: xDollar: ['Liquity']: 301514.4344615274
CDP: PolyQuity: ['Liquity']: 195266.66313958424
CDP: TopShelf: ['Liquity']: 129433.46417120536
CDP: MahaDAO Arth: ['Liquity']: 51485.05351558609
CDP: Teddy Cash: ['Liquity']: 49517.36
CDP: Fluity: ['Liquity']: 37984.00263582736
CDP: Stabiliz

,category,largest_protocols,percent_tvl_largest_protocols,percent_tvl_largest_protocols_and_forks,total_tvl_largest_protocols,total_tvl_largest_protocols_and_forks,fork_protocols,total_tvl_fork_protocols
0,CEX,"['Binance CEX', 'OKX', 'Bitfinex', 'Crypto-com...",85.45%,85.45%,$84427.83M,$84427.83M,[],$0.0M
1,Liquid Staking,"['Lido', 'Coinbase Wrapped Staked ETH', 'Rocke...",91.47%,91.96%,$17890.06M,$17985.56M,['OKC Liquid Staking'],$95.5M
2,CDP,"['MakerDAO', 'JustStables', 'Liquity', 'Abraca...",92.68%,94.0%,$8629.27M,$8752.71M,"['Helio Protocol', 'Gravita Protocol', 'DeFi F...",$123.44M
3,Chain,"['Polygon Bridge & Staking', 'Stacks Staking',...",99.96%,99.96%,$6486.39M,$6486.39M,[],$0.0M
4,Bridge,"['WBTC', 'JustCryptos', 'Multichain', 'Portal'...",90.24%,90.24%,$9412.66M,$9412.66M,[],$0.0M
5,Dexes,"['Curve DEX', 'Uniswap V3', 'PancakeSwap AMM',...",63.0%,83.85%,$10359.96M,$13788.75M,"['PancakeSwap AMM', 'SushiSwap', 'PancakeSwap ...",$3428.79M
6,Lending,"['AAVE V2', 'JustLend', 'AAVE V3', 'Compound',...",78.13%,87.39%,$10900.66M,$12192.63M,"['Venus', 'Benqi Lending', 'Tectonic', 'Flux F...",$1291.97M
7,Yield,"['Convex Finance', 'Aura', 'Coinwind', 'Conic ...",81.53%,82.2%,$4112.8M,$4146.72M,"['Wombex Finance', 'Quoll', 'Muuu Finance']",$33.92M
8,Services,"['Instadapp', 'DefiSaver', 'CIAN', 'SPL Govern...",98.12%,98.12%,$2304.29M,$2304.29M,[],$0.0M
9,Derivatives,"['GMX', 'dYdX', 'Gains Network', 'ApolloX', 'M...",78.89%,81.69%,$1063.46M,$1101.2M,"['Mummy Finance', 'QuickPerps', 'Alpaca Perpet...",$37.74M


In [62]:
# For all "orginal" protocols, find the sum of their TVL and their forks.

df = pd.read_csv('protocols_all_small.csv')

# create an empty DataFrame to store the results
results = pd.DataFrame(columns=['category', 'protocol_name', 'num_forks', 'total_tvl', 'tvl_orig', 'tvl_forks', 'forks_names'])


# iterate over each category
for category in df['category'].unique():
    # filter the protocols in the category where forkedFrom is empty
    orig_protocols = df[(df['category'] == category) & (df['forkedFrom'].isnull())]
    
     # find the sum of TVL for each orig_protocol and all fork_protocols that have its name in forkedFrom 
    for index, protocol in orig_protocols.iterrows():
        # find the sum of TVL for the protocol and all protocols that have its name in forkedFrom

        # get protocols in this category where forkedFrom is not empty 
        mask = df[(df['category'] == category) & (df['forkedFrom'].notna())]
        # get protocols in this category where forkedFrom is not empty and contains any of the 5 largest protocols
        mask = mask[mask['forkedFrom'].apply(lambda x: any(item for item in [protocol['name']] if item in x))]
        forks = mask

        # if forks is not empty, find the sum of tvl of all protocols in forks
        if len(forks) > 0:
            total_tvl_forks = forks['tvl'].sum()
            forks_names = forks['name'].tolist()
        else:
            total_tvl_forks = 0
            forks_names = []

        # calculate the total_tvl_orig_and_forks
        total_tvl_orig_and_forks = protocol['tvl'] + total_tvl_forks
        
        # fill results
        new_row = pd.DataFrame({
            'category': category,
            'protocol_name': protocol['name'],
            'num_forks': len(forks),
            'total_tvl': total_tvl_orig_and_forks,
            'tvl_orig': protocol['tvl'],
            'tvl_forks': total_tvl_forks,
            'forks_names': [forks_names]
        }, index=[0])
        results = pd.concat([results, new_row], ignore_index=True)

    #   
results
# save to csv
results.to_csv('forks_by_protocol.csv', index=False)

# pretty print results, all tvl values in millions of $
df = pd.read_csv('forks_by_protocol.csv')   


In [63]:
import plotly.express as px

# read in the data
df = pd.read_csv('forks_by_protocol.csv')
all_protocols = pd.read_csv('protocols_all_small.csv')

# Filter to just the most interesting categories 
df = df[df['category'].isin(['Dexes', 'Lending', 'Yield', 'Derivatives', 'CDP', 'Yield Aggregator'])]

# iterate over each category
for category in df['category'].unique():
    # filter the data for the category
    category_data = df[df['category'] == category]
    
    # sort the data by total TVL in descending order
    category_data = category_data.sort_values(by='total_tvl', ascending=False)
    
    # calculate the cumulative sum of total TVL
    category_data['cumulative_tvl'] = category_data['total_tvl'].cumsum()
    
    # filter the protocols that cover up to 95% of total TVL
    top_protocols = category_data[category_data['cumulative_tvl'] <= 0.95 * category_data['total_tvl'].sum()]
    
    # calculate the TVL of the "other" category
    other_tvl = category_data[category_data['cumulative_tvl'] > 0.95 * category_data['total_tvl'].sum()]['total_tvl'].sum()
    
    # add the top protocols to the results DataFrame
    results = pd.concat([top_protocols, pd.DataFrame({'protocol_name': ['other'], 'total_tvl': [other_tvl]})])
    
    # create a bar chart for the category, stacked bar for each protcol, stacks for tvl_orig and tvl_forks
    fig = px.bar(results, x='protocol_name', y=['tvl_orig', 'tvl_forks'], title=category, barmode='stack')

    # how many bars until we get past 90% of total TVL
    num_bars_80 = len(results[results['cumulative_tvl'] <= 0.8 * results['total_tvl'].sum()])
    num_bars_90 = len(results[results['cumulative_tvl'] <= 0.9 * results['total_tvl'].sum()])
    num_bars_95 = len(results[results['cumulative_tvl'] <= 0.95 * results['total_tvl'].sum()])


    # add a vertical line to the chart to show the 80, 90, 95% thresholds

    fig.add_vline(x=num_bars_80 - 0.5, line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x=num_bars_90 - 0.5, line_width=3, line_dash="dash", line_color="orange")
    fig.add_vline(x=num_bars_95 - 0.5, line_width=3, line_dash="dash", line_color="red")    

    # show the chart
    fig.show()

    # publish this chart on my plotly account so that it's publicly accessible


    # sanity checks
    #print ('results_tvl =' + str(results['total_tvl'].sum()/1000000))
    #print("all protocols tvl = " + str(all_protocols[all_protocols['category'] == category]['tvl'].sum()/1000000))   

    # print summary stats
    print (str(len(results)) + " total protocols")
    print (str(num_bars_80) + " protocols cover 80% of total TVL")
    print (str(num_bars_90) + " protocols cover 90% of total TVL")
    print (str(num_bars_95) + " protocols cover 95% of total TVL")

7 total protocols
1 protocols cover 80% of total TVL
3 protocols cover 90% of total TVL
6 protocols cover 95% of total TVL


33 total protocols
5 protocols cover 80% of total TVL
16 protocols cover 90% of total TVL
32 protocols cover 95% of total TVL


10 total protocols
3 protocols cover 80% of total TVL
5 protocols cover 90% of total TVL
9 protocols cover 95% of total TVL


26 total protocols
3 protocols cover 80% of total TVL
12 protocols cover 90% of total TVL
25 protocols cover 95% of total TVL


15 total protocols
4 protocols cover 80% of total TVL
9 protocols cover 90% of total TVL
14 protocols cover 95% of total TVL


17 total protocols
6 protocols cover 80% of total TVL
11 protocols cover 90% of total TVL
16 protocols cover 95% of total TVL


In [ ]:
# Possible data issues

# why no kwenta, ribbon finance, etc.?  missing from Defillama
# get the data live

# TODOs
# misses protocols that have forkedFrom but we don't have the orig protocol in our dataset
    # - update orig_protocols to also include forkedFrom that are not in our dataset
# this would miss nested protocols that are a fork of a fork, assuming orig protocol is not in forkedFrom - unlikely but possible



# do by chain


# which have subgraph available?


